In [23]:
import re
def process_and_solve_equation(equation):
    # Split the equation at '=' and work with the right side
    _, equation_right = equation.split('=')
    
    # Replace all alphabetic strings (variables) with '1'
    modified_equation = re.sub(r'[a-zA-Z]+', '1', equation_right)
    
    try:
        # Attempt to evaluate the modified equation
        eval(modified_equation)
        return True  # If evaluation succeeds, return True
    except:
        return False  # If there's an error during evaluation, return False

# Example usage
equation = "a=+5"
result = process_and_solve_equation(equation)
print(f"Equation can be solved: {result}")

Equation can be solved: True


In [60]:
import re

def check_three_or_more_operators(equation):
    # Improved pattern to correctly handle the '**-' exception
    pattern = r'(?:[+\-*/]{3,}(?<!\*\*\-))'

    if re.search(pattern, equation):
        return False
    return True

# Test the function
equation1 = "a=5+++(lemon--gin) + b"  # Should return False
equation2 = "a=5+--(lemon--gin) + b"  # Should return False
equation3 = "a=5**+(lemon--gin) + b"  # Should return True

print(check_three_or_more_operators(equation1))
print(check_three_or_more_operators(equation2))
print(check_three_or_more_operators(equation3))


False
False
False


In [65]:
import re

def check_double_op(equation):
    # Updated pattern to exclude '**-' and '**' from being considered as double operators.
    # It looks for double operators, excluding our specified exceptions.
    pattern = r'(?<=[\+\*/])\+|\+(?=[\+\*/])|(?<=[\*/])\*|(?<=/)/|(?<!\*\*)\*{2}(?![\+\*/\-])|(?<=\*\*)[\+\*/]|\-(?=\-[\+\*/])|(?<=[\+\*/])-(?=\-)'
    
    if re.search(pattern, equation):
        return False
    return True

# Test the function
equation1 = "a=5**-(lemon--gin) + b"  # Should return True, **- is an exception
equation2 = "a=5**+(lemon--gin) + b"  # Should return False, **+ is not an exception
equation3 = "a=5++(lemon--gin) + b"   # Should return False

print(check_double_op(equation1))
print(check_double_op(equation2))
print(check_double_op(equation3))


False
False
False


In [73]:
def check_operators(s):
    # Define the allowed exceptions
    exceptions = {"**", "*-", "**-", "--", "+-", "/-"}
    operators = {"+", "-", "*", "/", "**"}
    
    # Iterate through the string to check for side by side operators
    for i in range(len(s) - 1):
        if s[i] in operators:
            # Check for two-character exceptions
            if s[i:i+2] in exceptions or s[i:i+3] in exceptions:
                continue
            # Check for a non-exception side by side operators
            if s[i+1] in operators:
                return False
    return True

# Test cases
test_cases = ["5 -+ 6", "7 ** 8", "9 --", "10 +- 2", "3 */ 2", "4 ++ 5"]

# Run tests
results = {case: check_operators(case) for case in test_cases}

results


{'5 -+ 6': False,
 '7 ** 8': True,
 '9 --': True,
 '10 +- 2': True,
 '3 */ 2': False,
 '4 ++ 5': False}

In [1]:

def is_valid_assignment(assignment_string):
    def is_numeric_equation(equation):
        numeric_pattern = r'^\s*[a-zA-Z_][a-zA-Z0-9_]*\s*=\s*\(\s*(?:\d*\.?\d+|\d+\.\d*|[a-zA-Z_][a-zA-Z0-9_]*)(?:\s*[\+\-\*\/]\s*(?:\d*\.?\d+|\d+\.\d*|[a-zA-Z_][a-zA-Z0-9_]*))*\s*\)\s*$'
        return bool(re.match(numeric_pattern, equation))

    def is_complex_equation(equation):
        complex_pattern = r'^\s*[a-zA-Z_][a-zA-Z0-9_]*\s*=\s*\((?:[a-zA-Z_0-9\.\+\-\*\/\s\(\)]+)\)\s*$'
        return bool(re.match(complex_pattern, equation))

    if not assignment_string:
        return False, "Input Error: Assignment cannot be empty"
    elif "=" not in assignment_string:
        return False, "Input Error: Assignment must contain '='"
    elif assignment_string.count("(") != assignment_string.count(")"):
        return False, "Input Error: Unequal number of opening and closing parentheses"
    else:
        assignment_string_clean = assignment_string.replace(" ", "")
        if is_numeric_equation(assignment_string_clean):
            return True, None
        elif is_complex_equation(assignment_string_clean):
            return True, None
        else:
            return False, "Input Error: Invalid Format"

In [5]:
is_valid_assignment("a=1+1") #True
is_valid_assignment("b=1+1")

NameError: name 're' is not defined

In [5]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        # Variable criteria remain the same, identifying alphabetic chars
        return char.isalpha()

    def _isoperator(self, char):
        # Operator criteria
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif exp[i] == '-':
                if (i != 0 and (exp[i - 1]).isdigit() or self._isvariable(exp[i - 1]) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]) or self._isoperator(exp[i + 1])):
                    tokens.append(exp[i])
                    i += 1
                else:
                    if (exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]):
                        tokens.extend(['(', exp[i + 1], '*', '-1', ')'])
                        i += 2
                    elif exp[i + 1] == '(':
                        tokens.append('(')
                        j = i + 1
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            j += 1
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j])
                        tokens.extend(['(', *inside_tokens, ')', '*', '-1', ')'])
                        i = j + 1
                    else:
                        print("Error")
                        break
            elif self._isvariable(exp[i]):
                # Start grouping variable chars
                var_start = i
                while i < len(exp) and self._isvariable(exp[i]):
                    i += 1
                # Append the entire variable as one token
                tokens.append(exp[var_start:i])
                continue  # Skip the next else block after variable
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(',expression.replace(" ", ""),')'))
        return self._tokenize_inner(exp)

# Example usage
tkn = ExpressionTokenizer()
print(tkn.tokenize("(-lemon+20)"))


['(', '(', '(', 'l', '*', '-1', ')', 'emon', '+', '2', '0', ')', ')']


In [6]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        return char.isalpha()

    def _isoperator(self, char):
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif self._isvariable(exp[i]):
                variable_name = exp[i]
                i += 1
                while i < len(exp) and self._isvariable(exp[i]):
                    variable_name += exp[i]
                    i += 1
                tokens.append(variable_name)
            elif exp[i].isdigit():
                number = exp[i]
                i += 1
                while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                    number += exp[i]
                    i += 1
                tokens.append(number)
            elif exp[i] == '-':
                if i == 0 or exp[i - 1] in {'+', '-', '*', '/', '(', '**'}:
                    # Negation or subtraction at the start or after an operator or opening bracket
                    if i + 1 < len(exp) and exp[i + 1] == '(':
                        tokens.append('(')  # Start of negated expression
                        j = i + 2
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                            j += 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j-1])
                        tokens.extend(inside_tokens + ['*', '-1'])  # Apply negation inside the brackets
                        tokens.append(')')
                        i = j  # Move past the processed section
                    else:
                        # Negation directly applied to a variable or number
                        tokens.append('(-1')
                        tokens.append('*')
                        i += 1
                        continue  # Skip the increment at the end to process the next character
                else:
                    tokens.append('-')  # Subtraction operation
                i += 1
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(', expression.replace(" ", ""), ')'))
        return self._tokenize_inner(exp)

tkn = ExpressionTokenizer()
tokens = tkn.tokenize("(1-(20*b)*(-a+3))")
print(tokens)


['(', '(', '1', '-', '(', '20', '*', 'b', ')', '*', '(', '(-1', '*', 'a', '+', '3', ')', ')', ')']


In [16]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        return char.isalpha()

    def _isoperator(self, char):
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif self._isvariable(exp[i]):
                variable_name = exp[i]
                i += 1
                while i < len(exp) and self._isvariable(exp[i]):
                    variable_name += exp[i]
                    i += 1
                tokens.append(variable_name)
            elif exp[i].isdigit():
                number = exp[i]
                i += 1
                while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                    number += exp[i]
                    i += 1
                tokens.append(number)
            elif exp[i] == '-':
                # Check if it's a normal minus sign based on your provided condition
                if (i != 0 and (exp[i - 1]).isdigit() or self._isvariable(exp[i - 1]) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]) or exp[i + 1] == '('):
                    tokens.append('-')  # Append '-' directly as an operator
                    i += 1
                else:
                    # Handle as negation
                    if exp[i + 1] == '(':
                        j = i + 2
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                            j += 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j-1])
                        tokens.extend(['(', *inside_tokens, '*', '-1', ')'])
                        i = j
                    elif exp[i + 1].isdigit() or self._isvariable(exp[i + 1]):
                        # For negative numbers or variables next
                        i += 1  # Move past '-' to process the number/variable
                        if exp[i].isdigit():  # Negative number
                            number = exp[i]
                            i += 1
                            while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                                number += exp[i]
                                i += 1
                            tokens.extend(['(', number, '*', '-1', ')'])
                        else:  # Negative variable
                            variable_name = exp[i]
                            i += 1
                            while i < len(exp) and self._isvariable(exp[i]):
                                variable_name += exp[i]
                                i += 1
                            tokens.extend(['(', variable_name, '*', '-1', ')'])
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(', expression.replace(" ", ""), ')'))
        return self._tokenize_inner(exp)

tkn = ExpressionTokenizer()
tokens = tkn.tokenize("1+-(20**-b)*(-a+3)")
print(tokens)


['(', '1', '+', '(', '20', '**', '(', 'b', '*', '-1', ')', '*', '-1', ')', '*', '(', '(', 'a', '*', '-1', ')', '+', '3', ')', ')']


In [14]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        return char.isalpha()

    def _isoperator(self, char):
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif self._isvariable(exp[i]):
                variable_name = exp[i]
                i += 1
                while i < len(exp) and self._isvariable(exp[i]):
                    variable_name += exp[i]
                    i += 1
                tokens.append(variable_name)
            elif exp[i].isdigit():
                number = exp[i]
                i += 1
                while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                    number += exp[i]
                    i += 1
                tokens.append(number)
            elif exp[i] == '-':
                # Check for double minus (normal minus followed by unary minus)
                if i + 1 < len(exp) and exp[i + 1] == '-':
                    # Skip the first '-', handle the second '-' as unary negation
                    i += 1  # Move to the second '-'
                    if exp[i + 1] == '(':
                        # Unary negation before a parenthesis
                        j = i + 2
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                            j += 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j-1])
                        tokens.extend(['-', '(', *inside_tokens, '*', '-1', ')'])  # Apply unary negation
                        i = j
                    else:
                        tokens.append('-')  # Treat the first '-' as normal, the second will be handled accordingly
                        i += 1
                elif (i != 0 and (exp[i - 1]).isdigit() or self._isvariable(exp[i - 1]) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]) or exp[i + 1] == '('):
                    tokens.append('-')  # Normal minus
                    i += 1
                else:
                    # Unary negation
                    if exp[i + 1] == '(':
                        j = i + 1
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                            j += 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j-1])
                        tokens.extend(['(', *inside_tokens, '*', '-1', ')'])
                        i = j
                    else:
                        i += 1  # This handles a unary minus directly before a variable/number
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(', expression.replace(" ", ""), ')'))
        return self._tokenize_inner(exp)

tkn = ExpressionTokenizer()
tokens = tkn.tokenize("1--(20*b)*(-a+3)")
print(tokens)


['(', '1', '-', '(', '20', '*', 'b', '*', '-1', ')', '*', '(', 'a', '+', '3', ')', ')']
